# Import modules and functions

In [ ]:
import os
import subprocess
from owslib.wfs import WebFeatureService
import shapely.wkt
import geopandas as gpd
import json
from pathlib import Path
import urllib
import urllib.request
import gzip
import pandas as pd
import fiona  # Importing fiona to handle geospatial data
import sys
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
import seaborn as sns
import mapclassify
import folium




# Function to install a package
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Ensure required packages are installed
try:
    import tqdm
except ImportError as e:
    package = str(e).split()[-1]
    install(package)

import warnings
# Ignore FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_columns', None)  # Show all columns of datasets



## File Paths

In [ ]:
# Step 2: Define File Paths
print("Step 2: Define File Paths")
municipality_workspace = r"C:\Users\daphn\Documents\MADE_THESIS\DATA\RAW_DATA\AMSTERDAM_MUNICIPALITY\AmsterdamMunicipality.shp"
air_quality_folder = r"C:\Users\daphn\Documents\MADE_THESIS\DATA\RAW_DATA\DATA_AIR_QUALITY\Clean_AMS_DataMixed_3Pollutants"
NO2_file = os.path.join(air_quality_folder, 'Clean_AMS_DataMixed_NO2_5Jul.shp')
tree_canopy_path = r"C:\Users\daphn\Documents\MADE_THESIS\DATA\CLEAN_DATA\Clipped_Tree_Data.shp"
output_folder = r"C:\Users\daphn\Documents\MADE_THESIS\DATA\CLEAN_DATA\NO2_DISTRICT_TRAFFIC_ANALYSIS"
district_shp_folder = r"C:\Users\daphn\Documents\MADE_THESIS\DATA\CLEAN_DATA\DISTRICTS_SHAPES"


In [ ]:
# Step 3: Load Shapefiles
print("Step 3: Load Shapefiles")
try:
    municipality_boundary = gpd.read_file(municipality_workspace)
    NO2_data = gpd.read_file(NO2_file)
    tree_canopy_data = gpd.read_file(tree_canopy_path)
    print("Shapefiles loaded successfully")
except Exception as e:
    print(f"Error loading shapefiles: {e}")
    sys.exit(1)

In [ ]:
# Step 4: Ensure Coordinate Systems Match
print("Step 4: Ensure Coordinate Systems Match")
try:
    municipality_crs = municipality_boundary.crs

    if NO2_data.crs != municipality_crs:
        NO2_data = NO2_data.to_crs(municipality_crs)
        print("NO2 data reprojected to match municipality CRS")

    if tree_canopy_data.crs != municipality_crs:
        tree_canopy_data = tree_canopy_data.to_crs(municipality_crs)
        print("Tree canopy data reprojected to match municipality CRS")

    print("Coordinate systems verified and matched")
except Exception as e:
    print(f"Error in checking/reprojecting CRS: {e}")
    sys.exit(1)

## Load study area and data

In [ ]:
# Step 5: Clip NO2 Data to the Municipality Boundary
print("Step 5: Clip NO2 data to the municipality boundary")
try:
    clipped_NO2_data = gpd.clip(NO2_data, municipality_boundary)
    clipped_NO2_data.to_file(os.path.join(output_folder, "clipped_NO2_data.shp"))
    print("NO2 data clipped successfully")
except Exception as e:
    print(f"Error in clipping NO2 data: {e}")
    sys.exit(1)


## Buffer

In [ ]:
# Step 6: Buffer the NO2 data
print("Step 6: Buffer the NO2 data")
try:
    buffer_distance = 15  # Buffer distance in meters
    clipped_NO2_data['geometry'] = clipped_NO2_data.geometry.buffer(buffer_distance, cap_style='flat')
    clipped_NO2_data['shp_area_bf'] = clipped_NO2_data.geometry.area
    clipped_NO2_data.to_file(os.path.join(output_folder, "buffered_NO2_data.shp"))
    print("Buffer creation completed")
except Exception as e:
    print(f"Error in creating buffers: {e}")
    sys.exit(1)

In [ ]:
from tqdm import tqdm
# Step 7: Process each buffer to identify trees within them and calculate the fraction of tree coverage
print("Step 7: Identify trees within buffers and calculate sum of tree areas")
processed_buffers = []

def process_buffer(buffer, tree_canopy, original_crs, visualize=False):
    try:
        # Use the original CRS from the GeoDataFrame to ensure the CRS matches
        if tree_canopy.crs != original_crs:
            tree_canopy = tree_canopy.to_crs(original_crs)
            print("Reprojected tree canopy data to match buffer CRS")

        # Use a bounding box to filter trees and create a spatial index
        buffer_bbox = buffer.geometry.bounds
        filtered_trees = tree_canopy.cx[buffer_bbox[0]:buffer_bbox[2], buffer_bbox[1]:buffer_bbox[3]]
        
        # Build a spatial index for the filtered trees
        sindex = filtered_trees.sindex
        
        # Filter further using spatial index and buffer intersection
        possible_matches_index = list(sindex.intersection(buffer.geometry.bounds))
        possible_matches = filtered_trees.iloc[possible_matches_index]
        trees_within_buffer = possible_matches[possible_matches.intersects(buffer.geometry)]

        # Perform a precise intersection to get only the intersecting parts
        if not trees_within_buffer.empty:
            # Clip the tree geometries to the buffer to get only the intersecting parts
            trees_within_buffer['geometry'] = trees_within_buffer['geometry'].apply(lambda geom: geom.intersection(buffer.geometry))

            # Calculate the area of the clipped geometries
            intersection_area = trees_within_buffer.geometry.area.sum()

            # Debugging: Print out the areas and geometry types to ensure correctness
            print(f"Buffer ID: {buffer.name}")
            print(f"Buffer Area: {buffer['shp_area_bf']}")
            print(f"Intersection Area (Tree Coverage): {intersection_area}")
            print(f"Calculated Fraction: {intersection_area / buffer['shp_area_bf']}")
            print(f"Tree Geometry Types after Intersection: {trees_within_buffer.geometry.type.unique()}")
        else:
            intersection_area = 0

        # Calculate the fraction of the buffer covered by tree canopy
        buffer['sum_area_tb'] = intersection_area
        buffer['frac_area_tb'] = intersection_area / buffer['shp_area_bf']

        # If frac_area_tb is greater than 1, log a warning
        if buffer['frac_area_tb'] > 1:
            print(f"Warning: frac_area_tb exceeded 1 for buffer at index {buffer.name}. Inspect the data.")
            return None

        # Visualization: plot the buffer and intersecting trees
        if visualize:
            fig, ax = plt.subplots()
            buffer_gdf = gpd.GeoDataFrame([buffer], crs=tree_canopy.crs)
            buffer_gdf.plot(ax=ax, facecolor='none', edgecolor='blue', linewidth=2, label='Buffer')
            trees_within_buffer.plot(ax=ax, color='green', alpha=0.5, label='Intersecting Trees')
            plt.legend()
            plt.title(f"Buffer {buffer.name} and Intersecting Trees")
            plt.show()

        return buffer

    except Exception as e:
        print(f"Error processing buffer: {e}")
        return None

# Process all buffered NO2 data
for idx, buffer in tqdm(clipped_NO2_data.iterrows(), total=clipped_NO2_data.shape[0]):
    buffer_result = process_buffer(buffer, tree_canopy_data, clipped_NO2_data.crs, visualize=(idx == 0))  # Visualize the first buffer
    if buffer_result is not None:
        processed_buffers.append(buffer_result)


In [ ]:
# Step 8: Combine processed buffers and save results
if processed_buffers:
    processed_buffers_gdf = gpd.GeoDataFrame(processed_buffers, crs=clipped_NO2_data.crs)
    processed_buffers_gdf.to_file(os.path.join(output_folder, "processed_buffers.shp"))
    print("Processed buffers successfully and saved")
else:
    print("No buffers were processed successfully")

# Choose the index of the buffer you want to visualize (e.g., the second buffer)
buffer_index = 35050

# Index starts from 0, so 1 means the second buffer
buffer_to_visualize = processed_buffers[buffer_index]

# Proceed with the visualization
buffer_geometry = buffer_to_visualize['geometry']
intersecting_trees = tree_canopy_data[tree_canopy_data.intersects(buffer_geometry)]
intersecting_trees['geometry'] = intersecting_trees['geometry'].apply(lambda geom: geom.intersection(buffer_geometry))

fig, ax = plt.subplots()
gpd.GeoDataFrame([buffer_to_visualize], crs=tree_canopy_data.crs).plot(ax=ax, facecolor='none', edgecolor='blue', linewidth=2, label='Buffer')
intersecting_trees.plot(ax=ax, color='green', alpha=0.5, label='Intersecting Trees')
plt.legend()
plt.title(f'Buffer {buffer_index + 1} and Intersecting Trees')
plt.show()


In [ ]:
#

## Visualization and Analysis (including scatter plots)

In [ ]:
# Step 9: Visualization and Analysis (including scatter plots)

# Calculate and print the correlation
correlation = processed_buffers_gdf['frac_area_tb'].corr(processed_buffers_gdf['NO2_Data'])
print(f"Correlation: {correlation}")


In [ ]:
# Linear Regression Analysis
X = processed_buffers_gdf[['frac_area_tb']].values.reshape(-1, 1)
y = processed_buffers_gdf['NO2_Data'].values
model = LinearRegression()
model.fit(X, y)
trend_line = model.predict(X)  # Generate the predicted values for the trend line


In [ ]:
# Step 9: Visualization and Analysis (including scatter plots)

# Step 9: Visualization and Analysis (including scatter plots and histogram)

# Calculate and print the correlation
correlation = processed_buffers_gdf['frac_area_tb'].corr(processed_buffers_gdf['NO2_Data'])
print(f"Correlation: {correlation}")

# Linear Regression Analysis
X = processed_buffers_gdf[['frac_area_tb']].values.reshape(-1, 1)
y = processed_buffers_gdf['NO2_Data'].values
model = LinearRegression()
model.fit(X, y)
trend_line = model.predict(X)  # Generate the predicted values for the trend line

# Scatter Plot with Trend Line
plt.figure(figsize=(10, 6))
plt.scatter(processed_buffers_gdf['frac_area_tb'], processed_buffers_gdf['NO2_Data'], alpha=0.5, label='Data Points')

sorted_indices = np.argsort(processed_buffers_gdf['frac_area_tb'])
sorted_frac_area_tb = processed_buffers_gdf['frac_area_tb'].values[sorted_indices]
sorted_trend_line = trend_line[sorted_indices]

# Plotting the trend line
plt.plot(sorted_frac_area_tb, sorted_trend_line, color='red', label='Trend Line')

plt.title('Scatter Plot of Tree Fraction vs. NO2 Data')
plt.xlabel('Fraction of Trees')
plt.ylabel('NO2 Data')
plt.grid()
plt.legend()

# Save the scatter plot as PNG
output_scatter_path = os.path.join(output_folder, "scatter_plot_tree_vs_NO2.png")
plt.savefig(output_scatter_path)
print(f"Scatter Plot saved: {output_scatter_path}")

plt.show()

# Histogram of NO2 Data
plt.figure(figsize=(10, 6))
plt.hist(processed_buffers_gdf['NO2_Data'], bins=30, alpha=0.7, label='NO2 Data')
plt.xlabel('NO2 Concentration')
plt.ylabel('Frequency')
plt.title('Histogram of NO2 Data')

# Save the histogram as PNG
output_histogram_path = os.path.join(output_folder, "histogram_NO2_data.png")
plt.savefig(output_histogram_path)
print(f"Histogram saved: {output_histogram_path}")

plt.grid(True)
plt.legend()
plt.show()


In [ ]:
#run 

In [ ]:
import geopandas as gpd
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

# Define file paths
traffic_data_path = r"C:\Users\daphn\Documents\MADE_THESIS\DATA\RAW_DATA\DATA_TRAFFIC\segment_noord-holland_p20008_mt2021_j2020_rpgw_s-compact (3)\segment_noord-holland_p20008.shp"
processed_buffers_path = r"C:\Users\daphn\Documents\MADE_THESIS\DATA\CLEAN_DATA\NO2_DISTRICT_TRAFFIC_ANALYSIS\processed_buffers.shp"
output_folder = r"C:\Users\daphn\Documents\MADE_THESIS\DATA\CLEAN_DATA\NO2_DISTRICT_TRAFFIC_ANALYSIS"

district_shp_folder = r"C:\Users\daphn\Documents\MADE_THESIS\DATA\CLEAN_DATA\DISTRICTS_SHAPES"

# Load traffic data
traffic_data = gpd.read_file(traffic_data_path)
print(f"Traffic data loaded successfully. Columns: {traffic_data.columns}")

# Load processed buffers data
processed_buffers_gdf = gpd.read_file(processed_buffers_path)
print(f"Processed buffers data loaded successfully. Columns: {processed_buffers_gdf.columns}")

# Check CRS for both datasets
if traffic_data.crs != processed_buffers_gdf.crs:
    traffic_data = traffic_data.to_crs(processed_buffers_gdf.crs)
    print("Traffic data reprojected to match processed buffers CRS.")

In [ ]:
# Function to classify traffic into quartiles
def classify_into_quartiles(traffic_data, traffic_column='INT_LV'):
    if traffic_column in traffic_data.columns:
        traffic_data['Quartile'] = pd.qcut(traffic_data[traffic_column], 4, labels=['Bottom 25%', '25%-50%', '50%-75%', 'Top 25%'])
        print("Traffic data classified into quartiles.")
    else:
        print(f"Traffic column '{traffic_column}' not found.")
    return traffic_data

In [ ]:
# Function to perform spatial join between traffic and buffer data
def perform_spatial_join(traffic_data, processed_buffers_gdf):
    spatial_joined_gdf = gpd.sjoin(processed_buffers_gdf, traffic_data, how='inner', predicate='intersects')
    print("Spatial join completed.")
    return spatial_joined_gdf

In [ ]:
import pandas as pd

# Define the traffic column to classify
traffic_column = 'INT_LV'

# Check if the traffic column exists
if traffic_column in traffic_data.columns:
    # Classify the traffic data into quartiles based on the traffic column
    traffic_data['Quartile'] = pd.qcut(traffic_data[traffic_column], 4, labels=['Bottom 25%', '25%-50%', '50%-75%', 'Top 25%'])
    print("Traffic data divided into quartiles. First few rows with Quartiles:")
    print(traffic_data[['SEGMENT_ID', traffic_column, 'Quartile']].head())
else:
    print(f"Traffic column '{traffic_column}' not found in the dataset.")

In [ ]:
# Perform spatial join between processed buffers and traffic data
spatial_joined_gdf = gpd.sjoin(processed_buffers_gdf, traffic_data, how='inner', predicate='intersects')

# Check the result of the spatial join
print("Spatial join completed. First few rows of the joined dataset:")
print(spatial_joined_gdf.head())

# List the columns in the joined GeoDataFrame to confirm
print("Columns after spatial join:", spatial_joined_gdf.columns)

In [ ]:
# Function to clip the data to the district boundary
def clip_to_boundary(gdf, district_boundary):
    clipped_gdf = gpd.clip(gdf, district_boundary)
    return clipped_gdf

In [ ]:
# Function to create and save maps for top 25% and bottom 25% traffic segments
def create_traffic_map(district_name, district_boundary, top_25_gdf, bottom_25_gdf, output_folder):
    # Create a plot for the district
    fig, ax = plt.subplots(figsize=(10, 10))

    # Plot the district boundary
    district_boundary.plot(ax=ax, color='none', edgecolor='black', linewidth=2, label='District Boundary')

    # Plot top 25% traffic data in red
    if not top_25_gdf.empty:
        top_25_gdf.plot(ax=ax, color='red', label='Top 25% Traffic', alpha=0.7)

    # Plot bottom 25% traffic data in blue
    if not bottom_25_gdf.empty:
        bottom_25_gdf.plot(ax=ax, color='blue', label='Bottom 25% Traffic', alpha=0.7)

    # Set axis limits to the district boundary
    ax.set_xlim(district_boundary.total_bounds[0], district_boundary.total_bounds[2])
    ax.set_ylim(district_boundary.total_bounds[1], district_boundary.total_bounds[3])

    # Add labels and title
    plt.title(f"Top and Bottom 25% Traffic in {district_name}")
    plt.legend()
    plt.grid(True)

    # Save the plot
    map_output_path = os.path.join(output_folder, f"{district_name}_traffic_map.png")
    plt.savefig(map_output_path, dpi=300)
    plt.show()

    print(f"Map saved for {district_name} at {map_output_path}")

In [ ]:
print("Columns in processed_buffers_gdf:", processed_buffers_gdf.columns)
print("Columns in spatial_joined_gdf after join:", spatial_joined_gdf.columns)

In [ ]:
import os
import matplotlib.pyplot as plt
import geopandas as gpd
from sklearn.linear_model import LinearRegression

# Function to create and save scatter plots for top and bottom 25% traffic segments
def save_and_plot_top_bottom(gdf, data_column, title, output_folder):
    # Filter for top 25% and bottom 25%
    top_25_gdf = gdf[gdf['Quartile'] == 'Top 25%']
    bottom_25_gdf = gdf[gdf['Quartile'] == 'Bottom 25%']
    
    def plot_data(filtered_gdf, subset_title):
        if 'frac_area_' not in filtered_gdf.columns or data_column not in filtered_gdf.columns:
            print(f"Required columns 'frac_area_' or '{data_column}' are missing from the dataset.")
            return

        # Scatter Plot with Trend Line
        plt.figure(figsize=(10, 6))
        plt.scatter(filtered_gdf['frac_area_'], filtered_gdf[data_column], alpha=0.5, label=f'{subset_title} Data Points')

        # Linear regression
        X = filtered_gdf[['frac_area_']].dropna()
        y = filtered_gdf[data_column].dropna()
        if not X.empty and not y.empty:
            model = LinearRegression()
            model.fit(X, y)
            trend_line = model.predict(X)
            plt.plot(X, trend_line, 'r--', label='Trend Line')
            correlation = X['frac_area_'].corr(y)
            coef = model.coef_[0]
            intercept = model.intercept_
            print(f"Correlation: {correlation}, Coefficient: {coef}, Intercept: {intercept}")
        else:
            print(f"No valid data for regression in {subset_title}")

        plt.xlabel('Fraction of Trees')
        plt.ylabel('NO2 Data µg/m3')
        plt.title(f'Scatter Plot of {data_column} ({subset_title})')
        plt.grid(True)
        plt.legend()

        # Save scatter plot
        scatter_png_path = os.path.join(output_folder, f"{subset_title.replace(' ', '_')}_scatter.png")
        plt.savefig(scatter_png_path, dpi=300)
        plt.show()

    # Plot for Top 25%
    plot_data(top_25_gdf, f"{title} - Top 25% Traffic")

    # Plot for Bottom 25%
    plot_data(bottom_25_gdf, f"{title} - Bottom 25% Traffic")

# Function to process each district and generate maps and scatter plots
def process_district_for_maps_and_plots(district_name, traffic_data, processed_buffers_gdf, output_folder):
    print(f"\nProcessing district: {district_name}")
    
    # Load district boundary shapefile
    district_shapefile = os.path.join(district_shp_folder, f"{district_name}.shp")
    district_boundary = gpd.read_file(district_shapefile)
    
    # Ensure CRS consistency (reproject if necessary)
    if traffic_data.crs != 'EPSG:28992':
        traffic_data = traffic_data.to_crs(epsg=28992)
    if processed_buffers_gdf.crs != 'EPSG:28992':
        processed_buffers_gdf = processed_buffers_gdf.to_crs(epsg=28992)
    if district_boundary.crs != 'EPSG:28992':
        district_boundary = district_boundary.to_crs(epsg=28992)

    # Clip traffic data and processed buffers to the district boundary
    traffic_data_clipped = gpd.clip(traffic_data, district_boundary)
    processed_buffers_clipped = gpd.clip(processed_buffers_gdf, district_boundary)
    
    # Classify traffic data into quartiles
    traffic_data_clipped = classify_into_quartiles(traffic_data_clipped, 'INT_LV')

    # Perform spatial join for the traffic and processed buffers
    spatial_joined_gdf = gpd.sjoin(traffic_data_clipped, processed_buffers_clipped, how="inner", predicate="intersects")

    # Filter for top 25% and bottom 25% traffic
    top_25_gdf = spatial_joined_gdf[spatial_joined_gdf['Quartile'] == 'Top 25%']
    bottom_25_gdf = spatial_joined_gdf[spatial_joined_gdf['Quartile'] == 'Bottom 25%']

    # Check if the data exists for both top and bottom 25%
    if top_25_gdf.empty:
        print(f"No top 25% traffic data available for {district_name}")
    if bottom_25_gdf.empty:
        print(f"No bottom 25% traffic data available for {district_name}")

    # Create and save the scatter plots for top and bottom 25%
    save_and_plot_top_bottom(spatial_joined_gdf, 'NO2_Data', district_name, output_folder)

# Define districts (update this with actual district names)
districts = ["Centrum", "Nieuw-West", "West", "Zuid", "Noord", "Oost", "Zuidoost"]

# Run for all districts
for district_name in districts:
    process_district_for_maps_and_plots(district_name, traffic_data, processed_buffers_gdf, output_folder)


In [ ]:
# Function to create and save maps showing top 25% and bottom 25% traffic data with color differentiation
def create_clipping_map_with_quartiles(district_name, district_boundary, spatial_joined_gdf, output_folder):
    # Create a plot for the district
    fig, ax = plt.subplots(figsize=(10, 10))

    # Plot the district boundary
    district_boundary.plot(ax=ax, color='none', edgecolor='black', linewidth=2, label='District Boundary')

    # Filter for top 25% and bottom 25% traffic
    top_25_gdf = spatial_joined_gdf[spatial_joined_gdf['Quartile'] == 'Top 25%']
    bottom_25_gdf = spatial_joined_gdf[spatial_joined_gdf['Quartile'] == 'Bottom 25%']

    # Plot top 25% traffic data in red
    if not top_25_gdf.empty:
        top_25_gdf.plot(ax=ax, color='red', label='Top 25% Traffic', alpha=0.7)

    # Plot bottom 25% traffic data in blue
    if not bottom_25_gdf.empty:
        bottom_25_gdf.plot(ax=ax, color='blue', label='Bottom 25% Traffic', alpha=0.7)

    # Set axis limits to the district boundary
    ax.set_xlim(district_boundary.total_bounds[0], district_boundary.total_bounds[2])
    ax.set_ylim(district_boundary.total_bounds[1], district_boundary.total_bounds[3])

    # Add labels and title
    plt.title(f"Top and Bottom 25% Traffic in {district_name}")
    plt.legend()
    plt.grid(True)

    # Save the plot
    map_output_path = os.path.join(output_folder, f"{district_name}_traffic_map_with_quartiles.png")
    plt.savefig(map_output_path, dpi=300)
    plt.show()

    print(f"Map with quartiles saved for {district_name} at {map_output_path}")

# Function to create and save combined violin and box plot, as well as a histogram for all traffic quartiles
def save_and_plot_statistical_analysis(gdf, data_column, title, output_folder):
    # Ensure the necessary columns exist
    if 'Quartile' not in gdf.columns or data_column not in gdf.columns:
        print(f"Error: Required columns 'Quartile' or '{data_column}' are missing from the dataset.")
        print(f"Columns available in the data: {gdf.columns}")
        return

    print(f"Generating plots for {title} using {data_column}...")

    # Combined Violin and Box Plot
    plt.figure(figsize=(10, 6))
    sns.violinplot(x='Quartile', y=data_column, data=gdf, inner=None, color='lightgray', alpha=0.6)
    sns.boxplot(x='Quartile', y=data_column, data=gdf, width=0.2, boxprops=dict(alpha=0.6))

    plt.title(f'Combined Violin and Box Plot of {data_column} ({title})')
    plt.xlabel('Traffic Quartile')
    plt.ylabel('NO2 Data µg/m3')
    plt.grid(True)

    combined_violin_boxplot_path = os.path.join(output_folder, f"{title.replace(' ', '_')}_combined_violin_boxplot.png")
    plt.savefig(combined_violin_boxplot_path, dpi=300)
    plt.show()
    print(f"Combined violin and box plot saved at: {combined_violin_boxplot_path}")

    # Histogram for each quartile
    plt.figure(figsize=(10, 6))
    sns.histplot(gdf, x=data_column, hue='Quartile', multiple='stack', bins=30)
    plt.title(f'Histogram of {data_column} by Quartile ({title})')
    plt.xlabel('NO2 Data µg/m3')
    plt.ylabel('Frequency')
    plt.grid(True)

    hist_png_path = os.path.join(output_folder, f"{title.replace(' ', '_')}_histogram.png")
    plt.savefig(hist_png_path, dpi=300)
    plt.show()
    print(f"Histogram saved at: {hist_png_path}")

    print(f"Statistical analysis saved for {title}")

# Function to process each district, generate maps, and save statistical plots
def process_district_for_analysis(district_name, traffic_data, processed_buffers_gdf, output_folder, data_column):
    print(f"\nProcessing district: {district_name}")
    
    # Load district boundary shapefile
    district_shapefile = os.path.join(district_shp_folder, f"{district_name}.shp")
    district_boundary = gpd.read_file(district_shapefile)
    
    # Ensure CRS consistency (reproject if necessary)
    if traffic_data.crs != 'EPSG:28992':
        traffic_data = traffic_data.to_crs(epsg=28992)
    if processed_buffers_gdf.crs != 'EPSG:28992':
        processed_buffers_gdf = processed_buffers_gdf.to_crs(epsg=28992)
    if district_boundary.crs != 'EPSG:28992':
        district_boundary = district_boundary.to_crs(epsg=28992)

    # Clip traffic data and processed buffers to the district boundary
    traffic_data_clipped = gpd.clip(traffic_data, district_boundary)
    processed_buffers_clipped = gpd.clip(processed_buffers_gdf, district_boundary)
    
    # Perform spatial join for the traffic and processed buffers
    spatial_joined_gdf = gpd.sjoin(traffic_data_clipped, processed_buffers_clipped, how="inner", predicate="intersects")

    # Generate the clipping map with quartiles
    create_clipping_map_with_quartiles(district_name, district_boundary, spatial_joined_gdf, output_folder)
    
    # Perform and save the statistical analysis
    save_and_plot_statistical_analysis(spatial_joined_gdf, data_column, district_name, output_folder)

# Define districts (update this with actual district names)
districts = ["Centrum", "Nieuw-West", "West", "Zuid", "Noord", "Oost", "Zuidoost"]

# Define data column for analysis (update this with the correct column name, e.g., 'NO2_Data')
data_column = 'NO2_Data'

# Run for all districts
for district_name in districts:
    process_district_for_analysis(district_name, traffic_data, processed_buffers_gdf, output_folder, data_column)


## Calculate # of streets

In [ ]:
# Function to calculate the percentage of streets in the top 25% and bottom 25% per district
def calculate_street_percentages(spatial_joined_gdf, district_name):
    total_streets = len(spatial_joined_gdf)
    top_25_streets = len(spatial_joined_gdf[spatial_joined_gdf['Quartile'] == 'Top 25%'])
    bottom_25_streets = len(spatial_joined_gdf[spatial_joined_gdf['Quartile'] == 'Bottom 25%'])

    # Calculate percentages
    top_25_percentage = (top_25_streets / total_streets) * 100 if total_streets > 0 else 0
    bottom_25_percentage = (bottom_25_streets / total_streets) * 100 if total_streets > 0 else 0

    print(f"{district_name} - Top 25% Streets: {top_25_percentage:.2f}% ({top_25_streets}/{total_streets})")
    print(f"{district_name} - Bottom 25% Streets: {bottom_25_percentage:.2f}% ({bottom_25_streets}/{total_streets})")
    return top_25_percentage, bottom_25_percentage

# Function to process each district and calculate street percentages
def process_district_for_street_analysis(district_name, traffic_data, processed_buffers_gdf, output_folder):
    print(f"\nProcessing district: {district_name}")
    
    # Load district boundary shapefile
    district_shapefile = os.path.join(district_shp_folder, f"{district_name}.shp")
    district_boundary = gpd.read_file(district_shapefile)
    
    # Ensure CRS consistency (reproject if necessary)
    if traffic_data.crs != 'EPSG:28992':
        traffic_data = traffic_data.to_crs(epsg=28992)
    if processed_buffers_gdf.crs != 'EPSG:28992':
        processed_buffers_gdf = processed_buffers_gdf.to_crs(epsg=28992)
    if district_boundary.crs != 'EPSG:28992':
        district_boundary = district_boundary.to_crs(epsg=28992)

    # Clip traffic data and processed buffers to the district boundary
    traffic_data_clipped = gpd.clip(traffic_data, district_boundary)
    processed_buffers_clipped = gpd.clip(processed_buffers_gdf, district_boundary)
    
    # Classify traffic data into quartiles
    traffic_data_clipped = classify_into_quartiles(traffic_data_clipped, 'INT_LV')

    # Perform spatial join for the traffic and processed buffers
    spatial_joined_gdf = gpd.sjoin(traffic_data_clipped, processed_buffers_clipped, how="inner", predicate="intersects")

    # Calculate street percentages for top and bottom 25%
    top_25_percentage, bottom_25_percentage = calculate_street_percentages(spatial_joined_gdf, district_name)

# Define districts (update this with actual district names)
districts = ["Centrum", "Nieuw-West", "West", "Zuid", "Noord", "Oost", "Zuidoost"]

# Run for all districts
for district_name in districts:
    process_district_for_street_analysis(district_name, traffic_data, processed_buffers_gdf, output_folder)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import os
import geopandas as gpd

# Function to create and save maps showing top 25% and bottom 25% traffic data with color differentiation
def create_clipping_map_with_quartiles(district_name, district_boundary, spatial_joined_gdf, output_folder):
    # Create a plot for the district
    fig, ax = plt.subplots(figsize=(10, 10))

    # Plot the district boundary
    district_boundary.plot(ax=ax, color='none', edgecolor='black', linewidth=2, label='District Boundary')

    # Filter for top 25% and bottom 25% traffic
    top_25_gdf = spatial_joined_gdf[spatial_joined_gdf['Quartile'] == 'Top 25%']
    bottom_25_gdf = spatial_joined_gdf[spatial_joined_gdf['Quartile'] == 'Bottom 25%']

    # Plot top 25% traffic data in red
    if not top_25_gdf.empty:
        top_25_gdf.plot(ax=ax, color='red', label='Top 25% Traffic', alpha=0.7)

    # Plot bottom 25% traffic data in blue
    if not bottom_25_gdf.empty:
        bottom_25_gdf.plot(ax=ax, color='blue', label='Bottom 25% Traffic', alpha=0.7)

    # Set axis limits to the district boundary
    ax.set_xlim(district_boundary.total_bounds[0], district_boundary.total_bounds[2])
    ax.set_ylim(district_boundary.total_bounds[1], district_boundary.total_bounds[3])

    # Add labels and title
    plt.title(f"Top and Bottom 25% Traffic in {district_name}")
    plt.legend()
    plt.grid(True)

    # Save the plot
    map_output_path = os.path.join(output_folder, f"{district_name}_traffic_map_with_quartiles.png")
    plt.savefig(map_output_path, dpi=300)
    plt.show()

    print(f"Map with quartiles saved for {district_name} at {map_output_path}")

# Function to create and save box plot, violin plot, and histogram for all traffic quartiles
def save_and_plot_statistical_analysis(gdf, data_column, title, output_folder):
    # Ensure the necessary columns exist
    if 'Quartile' not in gdf.columns or data_column not in gdf.columns:
        print(f"Error: Required columns 'Quartile' or '{data_column}' are missing from the dataset.")
        print(f"Columns available in the data: {gdf.columns}")
        return

    print(f"Generating plots for {title} using {data_column}...")

    # Box Plot
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='Quartile', y=data_column, data=gdf)
    plt.title(f'Box Plot of {data_column} ({title})')
    plt.xlabel('Traffic Quartile')
    plt.ylabel(data_column)
    plt.grid(True)

    box_png_path = os.path.join(output_folder, f"{title.replace(' ', '_')}_box.png")
    plt.savefig(box_png_path, dpi=300)
    plt.show()
    print(f"Box plot saved at: {box_png_path}")

    # Violin Plot
    plt.figure(figsize=(10, 6))
    sns.violinplot(x='Quartile', y=data_column, data=gdf)
    plt.title(f'Violin Plot of {data_column} ({title})')
    plt.xlabel('Traffic Quartile')
    plt.ylabel(data_column)
    plt.grid(True)

    violin_png_path = os.path.join(output_folder, f"{title.replace(' ', '_')}_violin.png")
    plt.savefig(violin_png_path, dpi=300)
    plt.show()
    print(f"Violin plot saved at: {violin_png_path}")

    # Histogram for each quartile
    plt.figure(figsize=(10, 6))
    sns.histplot(gdf, x=data_column, hue='Quartile', multiple='stack', bins=30)
    plt.title(f'Histogram of {data_column} by Quartile ({title})')
    plt.xlabel(data_column)
    plt.ylabel('Frequency')
    plt.grid(True)

    hist_png_path = os.path.join(output_folder, f"{title.replace(' ', '_')}_histogram.png")
    plt.savefig(hist_png_path, dpi=300)
    plt.show()
    print(f"Histogram saved at: {hist_png_path}")

    print(f"Statistical analysis saved for {title}")

# Function to process each district, generate maps, and save statistical plots
def process_district_for_analysis(district_name, traffic_data, processed_buffers_gdf, output_folder, data_column):
    print(f"\nProcessing district: {district_name}")
    
    # Load district boundary shapefile
    district_shapefile = os.path.join(district_shp_folder, f"{district_name}.shp")
    district_boundary = gpd.read_file(district_shapefile)
    
    # Ensure CRS consistency (reproject if necessary)
    if traffic_data.crs != 'EPSG:28992':
        traffic_data = traffic_data.to_crs(epsg=28992)
    if processed_buffers_gdf.crs != 'EPSG:28992':
        processed_buffers_gdf = processed_buffers_gdf.to_crs(epsg=28992)
    if district_boundary.crs != 'EPSG:28992':
        district_boundary = district_boundary.to_crs(epsg=28992)

    # Clip traffic data and processed buffers to the district boundary
    traffic_data_clipped = gpd.clip(traffic_data, district_boundary)
    processed_buffers_clipped = gpd.clip(processed_buffers_gdf, district_boundary)
    
    # Perform spatial join for the traffic and processed buffers
    spatial_joined_gdf = gpd.sjoin(traffic_data_clipped, processed_buffers_clipped, how="inner", predicate="intersects")

    # Generate the clipping map with quartiles
    create_clipping_map_with_quartiles(district_name, district_boundary, spatial_joined_gdf, output_folder)
    
    # Perform and save the statistical analysis
    save_and_plot_statistical_analysis(spatial_joined_gdf, data_column, district_name, output_folder)

# Define districts (update this with actual district names)
districts = ["Centrum", "Nieuw-West", "West", "Zuid", "Noord", "Oost", "Zuidoost"]

# Define data column for analysis (update this with the correct column name, e.g., 'NO2_Data')
data_column = 'NO2_Data'

# Run for all districts
for district_name in districts:
    process_district_for_analysis(district_name, traffic_data, processed_buffers_gdf, output_folder, data_column)


In [ ]:
# Step 9: Visualization and Analysis (including scatter plots)

# Step 9: Visualization and Analysis (including scatter plots and histogram)

# Calculate and print the correlation
correlation = processed_buffers_gdf['frac_area_tb'].corr(processed_buffers_gdf['NO2_Mixed'])
print(f"Correlation: {correlation}")

# Linear Regression Analysis
X = processed_buffers_gdf[['frac_area_tb']].values.reshape(-1, 1)
y = processed_buffers_gdf['NO2_Mixed'].values
model = LinearRegression()
model.fit(X, y)
trend_line = model.predict(X)  # Generate the predicted values for the trend line

# Scatter Plot with Trend Line
plt.figure(figsize=(10, 6))
plt.scatter(processed_buffers_gdf['frac_area_tb'], processed_buffers_gdf['NO2_Mixed'], alpha=0.5, label='Data Points')

sorted_indices = np.argsort(processed_buffers_gdf['frac_area_tb'])
sorted_frac_area_tb = processed_buffers_gdf['frac_area_tb'].values[sorted_indices]
sorted_trend_line = trend_line[sorted_indices]

# Plotting the trend line
plt.plot(sorted_frac_area_tb, sorted_trend_line, color='red', label='Trend Line')

plt.title('Scatter Plot of Tree Fraction vs. NO2 Mixed')
plt.xlabel('Fraction of Trees')
plt.ylabel('NO2 Mixed')
plt.grid()
plt.legend()

# Save the scatter plot as PNG
output_scatter_path = os.path.join(output_folder, "scatter_plot_tree_vs_NO2_mixed.png")
plt.savefig(output_scatter_path)
print(f"Scatter Plot saved: {output_scatter_path}")

plt.show()

# Histogram of NO2 Data
plt.figure(figsize=(10, 6))
plt.hist(processed_buffers_gdf['NO2_Mixed'], bins=30, alpha=0.7, label='NO2 Data Mixed')
plt.xlabel('NO2 Concentration')
plt.ylabel('Frequency')
plt.title('Histogram of NO2 Data Mixed')

# Save the histogram as PNG
output_histogram_path = os.path.join(output_folder, "histogram_NO2_data_mixed.png")
plt.savefig(output_histogram_path)
print(f"Histogram saved: {output_histogram_path}")

plt.grid(True)
plt.legend()
plt.show()

In [ ]:
print("Step 9: Generate Histograms for the Whole City and Each District")

# Histogram for the whole city
plt.figure(figsize=(10, 6))
plt.hist(processed_buffers_gdf['frac_area_tb'], bins=30, alpha=0.7, label='Tree Fraction (City)')
plt.xlabel('Fraction of Tree Canopy Coverage')
plt.ylabel('Frequency')
plt.title('Histogram of Tree Canopy Coverage Fraction - Whole City')
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
# Heat Maps
def plot_heat_map(gdf, column, title, cmap='YlOrRd'):
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    gdf.plot(column=column, ax=ax, legend=True, cmap=cmap)
    ax.set_title(title)
    plt.show()

plot_heat_map(processed_buffers_gdf, 'NO2_Data', 'NO2 Concentration Heat Map')
plot_heat_map(processed_buffers_gdf, 'frac_area_tb', 'Tree Canopy Coverage Heat Map')


In [ ]:
# Box Plot
processed_buffers_gdf['canopy_category'] = pd.qcut(processed_buffers_gdf['frac_area_tb'], 4, labels=['Low', 'Medium', 'High', 'Very High'])
sns.boxplot(x='canopy_category', y='NO2_Data', data=processed_buffers_gdf)
plt.title('Box Plot of NO2 Concentration by Tree Canopy Coverage')
plt.xlabel('Tree Canopy Coverage Category')
plt.ylabel('NO2 Concentration')
plt.show()


In [ ]:
import time


# Step 10: Clip the City-Wide Results to Each District and Save
print("Step 10: Clip the city-wide results to each district boundary and save them")
districts = ["Centrum", "Nieuw-West", "West", "Zuid", "Noord", "Oost", "Zuidoost"]

for district_name in districts:
    print(f"Processing {district_name}...")
    start_time = time.time()

    # Load the district boundary
    district_shapefile = os.path.join(district_shp_folder, f"{district_name}.shp")
    district_boundary = gpd.read_file(district_shapefile)
    
    # Ensure both datasets have the same CRS
    if district_boundary.crs != processed_buffers_gdf.crs:
        district_boundary = district_boundary.to_crs(processed_buffers_gdf.crs)

    # Check and fix invalid geometries in the district boundary
    if not district_boundary.is_valid.all():
        print(f"Fixing invalid geometries for {district_name}")
        district_boundary = district_boundary.buffer(0)
    
    # Simplify the district boundary to improve performance (optional)
    district_boundary = district_boundary.simplify(0.001)  # Adjust tolerance value as needed
    
    # Clip the city-wide results to the district boundary
    try:
        district_results = gpd.clip(processed_buffers_gdf, district_boundary)
    except Exception as e:
        print(f"Error processing {district_name}: {e}")
        continue
    
    # Convert the categorical 'canopy_category' column to string (if it exists)
    if 'canopy_category' in district_results.columns:
        district_results['canopy_category'] = district_results['canopy_category'].astype(str)
    
    # Optionally, rename columns with shorter names to avoid truncation
    if 'frac_area_tb' in district_results.columns:
        district_results = district_results.rename(columns={'frac_area_tb': 'tree_frac', 'NO2_Data': 'NO2_Data'})

    # Save the clipped results
    district_output_path = os.path.join(output_folder, f"processed_buffers_{district_name}.shp")
    try:
        district_results.to_file(district_output_path)
    except Exception as e:
        print(f"Error saving results for {district_name}: {e}")
        continue
    
    print(f"Results clipped and saved for {district_name} in {time.time() - start_time:.2f} seconds")
    
    # Histogram for Tree Canopy Coverage in the district
    plt.figure(figsize=(10, 6))
    plt.hist(district_results['tree_frac'], bins=30, alpha=0.7, label=f'Tree Fraction ({district_name})')
    plt.xlabel('Fraction of Tree Canopy Coverage')
    plt.ylabel('Frequency')
    plt.title(f'Histogram of Tree Canopy Coverage Fraction - {district_name}')
    plt.grid(True)
    plt.legend()

    # Save the plot as PNG and show in Jupyter Notebook
    output_hist_tree_frac = os.path.join(output_folder, f"{district_name}_tree_canopy_hist.png")
    plt.savefig(output_hist_tree_frac)
    plt.show()

    # Histogram for NO2 values for each district
    plt.figure(figsize=(10, 6))
    plt.hist(district_results['NO2_Data'], bins=30, alpha=0.7, label=f'NO2 Values ({district_name})')
    plt.xlabel('NO2 Concentration')
    plt.ylabel('Frequency')
    plt.title(f'Histogram of NO2 Concentration - {district_name}')
    plt.grid(True)
    plt.legend()

    # Save the plot as PNG and show in Jupyter Notebook
    output_hist_NO2 = os.path.join(output_folder, f"{district_name}_NO2_hist.png")
    plt.savefig(output_hist_NO2)
    plt.show()

    print(f"Histograms saved for {district_name}: {output_hist_tree_frac} and {output_hist_NO2}")


In [ ]:
import time


# Step 10: Clip the City-Wide Results to Each District and Save
print("Step 10: Clip the city-wide results to each district boundary and save them")
districts = ["Centrum", "Nieuw-West", "West", "Zuid", "Noord", "Oost", "Zuidoost"]

for district_name in districts:
    print(f"Processing {district_name}...")
    start_time = time.time()

    # Load the district boundary
    district_shapefile = os.path.join(district_shp_folder, f"{district_name}.shp")
    district_boundary = gpd.read_file(district_shapefile)
    
    # Ensure both datasets have the same CRS
    if district_boundary.crs != processed_buffers_gdf.crs:
        district_boundary = district_boundary.to_crs(processed_buffers_gdf.crs)

    # Check and fix invalid geometries in the district boundary
    if not district_boundary.is_valid.all():
        print(f"Fixing invalid geometries for {district_name}")
        district_boundary = district_boundary.buffer(0)
    
    # Simplify the district boundary to improve performance (optional)
    district_boundary = district_boundary.simplify(0.001)  # Adjust tolerance value as needed
    
    # Clip the city-wide results to the district boundary
    try:
        district_results = gpd.clip(processed_buffers_gdf, district_boundary)
    except Exception as e:
        print(f"Error processing {district_name}: {e}")
        continue
    
    # Convert the categorical 'canopy_category' column to string (if it exists)
    if 'canopy_category' in district_results.columns:
        district_results['canopy_category'] = district_results['canopy_category'].astype(str)
    
    # Optionally, rename columns with shorter names to avoid truncation
    if 'frac_area_tb' in district_results.columns:
        district_results = district_results.rename(columns={'frac_area_tb': 'tree_frac', 'NO2_Mixed': 'NO2_Mixed'})

    # Save the clipped results
    district_output_path = os.path.join(output_folder, f"processed_buffers_{district_name}.shp")
    try:
        district_results.to_file(district_output_path)
    except Exception as e:
        print(f"Error saving results for {district_name}: {e}")
        continue
    
    print(f"Results clipped and saved for {district_name} in {time.time() - start_time:.2f} seconds")
    
    # Histogram for Tree Canopy Coverage in the district
    plt.figure(figsize=(10, 6))
    plt.hist(district_results['tree_frac'], bins=30, alpha=0.7, label=f'Tree Fraction ({district_name})')
    plt.xlabel('Fraction of Tree Canopy Coverage')
    plt.ylabel('Frequency')
    plt.title(f'Histogram of Tree Canopy Coverage Fraction - {district_name}')
    plt.grid(True)
    plt.legend()

    # Save the plot as PNG and show in Jupyter Notebook
    output_hist_tree_frac = os.path.join(output_folder, f"{district_name}_tree_canopy_hist.png")
    plt.savefig(output_hist_tree_frac)
    plt.show()

    # Histogram for NO2 values for each district
    plt.figure(figsize=(10, 6))
    plt.hist(district_results['NO2_Mixed'], bins=30, alpha=0.7, label=f'NO2  Mixed Values ({district_name})')
    plt.xlabel('NO2 Mixed Concentration')
    plt.ylabel('Frequency')
    plt.title(f'Histogram of NO2 Mixed Concentration - {district_name}')
    plt.grid(True)
    plt.legend()

    # Save the plot as PNG and show in Jupyter Notebook
    output_hist_NO2 = os.path.join(output_folder, f"{district_name}_NO2_mixed_hist.png")
    plt.savefig(output_hist_NO2)
    plt.show()

    print(f"Histograms saved for {district_name}: {output_hist_tree_frac} and {output_hist_NO2}")

In [ ]:
# Step 11: Create Scatter Plots for Each District
print("Step 11: Create scatter plots for each district")

def create_scatter_plot(district_name, processed_buffers_gdf):
    if processed_buffers_gdf.empty:
        print(f"No data available for district: {district_name}")
        return
    
    # Ensure consistent column names
    correlation = processed_buffers_gdf['tree_frac'].corr(processed_buffers_gdf['NO2_Data'])
    print(f"Correlation for {district_name}: {correlation}")

    X = processed_buffers_gdf[['tree_frac']].values.reshape(-1, 1)
    y = processed_buffers_gdf['NO2_Data'].values
    model = LinearRegression()
    model.fit(X, y)
    trend_line = model.predict(X)

    plt.figure(figsize=(10, 6))
    plt.scatter(processed_buffers_gdf['tree_frac'], processed_buffers_gdf['NO2_Data'], alpha=0.5, label='Data Points')

    sorted_indices = np.argsort(processed_buffers_gdf['tree_frac'])
    sorted_tree_frac = processed_buffers_gdf['tree_frac'].values[sorted_indices]
    sorted_trend_line = trend_line[sorted_indices]

    plt.plot(sorted_tree_frac, sorted_trend_line, color='red', label='Trend Line')

    plt.title(f'Scatter Plot of Tree Fraction vs. NO2 Data in {district_name}')
    plt.xlabel('Fraction of Trees')
    plt.ylabel('NO2 Data')
    plt.grid()
    plt.legend()

    # Save the scatter plot as PNG
    scatter_plot_path = os.path.join(output_folder, f"scatter_plot_{district_name}.png")
    plt.savefig(scatter_plot_path)

    # Display the scatter plot in the notebook
    plt.show()

# Generate scatter plots for each district
for district_name in districts:
    district_output_path = os.path.join(output_folder, f"processed_buffers_{district_name}.shp")
    district_results = gpd.read_file(district_output_path)

    # Ensure CRS consistency
    if district_results.crs != processed_buffers_gdf.crs:
        district_results = district_results.to_crs(processed_buffers_gdf.crs)

    create_scatter_plot(district_name, district_results)

In [ ]:
# Step 11: Create Scatter Plots for Each District
print("Step 11: Create scatter plots for each district")

def create_scatter_plot(district_name, processed_buffers_gdf):
    if processed_buffers_gdf.empty:
        print(f"No data available for district: {district_name}")
        return
    
    # Ensure consistent column names
    correlation = processed_buffers_gdf['tree_frac'].corr(processed_buffers_gdf['NO2_Mixed'])
    print(f"Correlation for {district_name}: {correlation}")

    X = processed_buffers_gdf[['tree_frac']].values.reshape(-1, 1)
    y = processed_buffers_gdf['NO2_Mixed'].values
    model = LinearRegression()
    model.fit(X, y)
    trend_line = model.predict(X)

    plt.figure(figsize=(10, 6))
    plt.scatter(processed_buffers_gdf['tree_frac'], processed_buffers_gdf['NO2_Mixed'], alpha=0.5, label='Data Points')

    sorted_indices = np.argsort(processed_buffers_gdf['tree_frac'])
    sorted_tree_frac = processed_buffers_gdf['tree_frac'].values[sorted_indices]
    sorted_trend_line = trend_line[sorted_indices]

    plt.plot(sorted_tree_frac, sorted_trend_line, color='red', label='Trend Line')

    plt.title(f'Scatter Plot of Tree Fraction vs. NO2 Data in {district_name}')
    plt.xlabel('Fraction of Trees')
    plt.ylabel('NO2 Data Mixed')
    plt.grid()
    plt.legend()

    # Save the scatter plot as PNG
    scatter_plot_path = os.path.join(output_folder, f"scatter_plot_mixed_{district_name}.png")
    plt.savefig(scatter_plot_path)

    # Display the scatter plot in the notebook
    plt.show()

# Generate scatter plots for each district
for district_name in districts:
    district_output_path = os.path.join(output_folder, f"processed_buffers_{district_name}.shp")
    district_results = gpd.read_file(district_output_path)

    # Ensure CRS consistency
    if district_results.crs != processed_buffers_gdf.crs:
        district_results = district_results.to_crs(processed_buffers_gdf.crs)

    create_scatter_plot(district_name, district_results)


In [ ]:
# Step 13: Visualize Clipped Results for Each District to Verify Clipping
print("Step 13: Visualize Clipped Results for Each District")

def visualize_clipping(district_name, district_boundary, district_results):
    # Plot the district boundary and the clipped NO2 data for visualization
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Plot the district boundary
    district_boundary.boundary.plot(ax=ax, color='blue', linewidth=2, label=f'{district_name} Boundary')
    
    # Plot the clipped NO2 results
    district_results.plot(ax=ax, column='NO2', cmap='YlOrRd', legend=True, alpha=0.6)
    
    plt.title(f'Clipping Visualization for {district_name}')
    plt.legend()
    plt.show()

    # Optionally save the visualization
    fig.savefig(os.path.join(output_folder, f"clipping_visualization_{district_name}.png"))
    print(f"Clipping visualization saved for {district_name}")

for district_name in districts:
    district_output_path = os.path.join(output_folder, f"processed_buffers_{district_name}.shp")
    district_results = gpd.read_file(district_output_path)

    # Load the district boundary again
    district_shapefile = os.path.join(district_shp_folder, f"{district_name}.shp")
    district_boundary = gpd.read_file(district_shapefile)
    
    # Ensure CRS consistency
    if district_boundary.crs != processed_buffers_gdf.crs:
        district_boundary = district_boundary.to_crs(processed_buffers_gdf.crs)
    
    # Visualize the clipping
    visualize_clipping(district_name, district_boundary, district_results)

In [ ]:


# Step 13: Create Box Plots for Each District
print("Step 13: Create Box Plots for Each District")

# Box Plot with Categorized Tree Fractions
def create_box_plot(district_name, processed_buffers_gdf):
    # Categorize the tree_frac into 4 quartiles
    processed_buffers_gdf['tree_frac_cat'] = pd.qcut(processed_buffers_gdf['tree_frac'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])

    plt.figure(figsize=(10, 6))
    sns.boxplot(x='tree_frac_cat', y='NO2_Data', data=processed_buffers_gdf)
    plt.title(f'Box Plot of Tree Fraction vs. NO2 Data in {district_name}')
    plt.xlabel('Tree Canopy Coverage Category')
    plt.ylabel('NO2 Data')
    plt.grid(True)

    # Save the plot as PNG
    output_box_plot_path = os.path.join(output_folder, f"{district_name}_box_plot_NO2.png")
    plt.savefig(output_box_plot_path)
    print(f"Box Plot saved for {district_name}: {output_box_plot_path}")

    # Show the plot in Jupyter Notebook
    plt.show()

# Generate box plot for each district
for district_name in districts:
    district_output_path = os.path.join(output_folder, f"processed_buffers_{district_name}.shp")
    district_results = gpd.read_file(district_output_path)
    
    # Ensure CRS consistency
    if district_results.crs != processed_buffers_gdf.crs:
        district_results = district_results.to_crs(processed_buffers_gdf.crs)

    create_box_plot(district_name, district_results)


In [ ]:


# Step 13: Create Box Plots for Each District
print("Step 13: Create Box Plots for Each District")

# Box Plot with Categorized Tree Fractions
def create_box_plot(district_name, processed_buffers_gdf):
    # Categorize the tree_frac into 4 quartiles
    processed_buffers_gdf['tree_frac_cat'] = pd.qcut(processed_buffers_gdf['tree_frac'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])

    plt.figure(figsize=(10, 6))
    sns.boxplot(x='tree_frac_cat', y='NO2_Mixed', data=processed_buffers_gdf)
    plt.title(f'Box Plot of Tree Fraction vs. NO2 Data in {district_name}')
    plt.xlabel('Tree Canopy Coverage Category')
    plt.ylabel('NO2 Data Mixed')
    plt.grid(True)

    # Save the plot as PNG
    output_box_plot_path = os.path.join(output_folder, f"{district_name}_box_plot_NO2_mixed.png")
    plt.savefig(output_box_plot_path)
    print(f"Box Plot saved for {district_name}: {output_box_plot_path}")

    # Show the plot in Jupyter Notebook
    plt.show()

# Generate box plot for each district
for district_name in districts:
    district_output_path = os.path.join(output_folder, f"processed_buffers_{district_name}.shp")
    district_results = gpd.read_file(district_output_path)
    
    # Ensure CRS consistency
    if district_results.crs != processed_buffers_gdf.crs:
        district_results = district_results.to_crs(processed_buffers_gdf.crs)

    create_box_plot(district_name, district_results)

In [ ]:
# Step 14: Create Violin Plots for Each District
print("Step 14: Create Violin Plots for Each District")

# Violin Plot with Categorized Tree Fractions
def create_violin_plot(district_name, processed_buffers_gdf):
    # Categorize the tree_frac into 4 quartiles
    processed_buffers_gdf['tree_frac_cat'] = pd.qcut(processed_buffers_gdf['tree_frac'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])

    plt.figure(figsize=(10, 6))
    sns.violinplot(x='tree_frac_cat', y='NO2_Data', data=processed_buffers_gdf, cut=0)
    plt.title(f'Violin Plot of Tree Fraction vs. NO2 Data in {district_name}')
    plt.xlabel('Tree Canopy Coverage Category')
    plt.ylabel('NO2 Data')
    plt.grid(True)

    # Save the plot as PNG
    output_violin_plot_path = os.path.join(output_folder, f"{district_name}_violin_plot_NO2.png")
    plt.savefig(output_violin_plot_path)
    print(f"Violin Plot saved for {district_name}: {output_violin_plot_path}")

    # Show the plot in Jupyter Notebook
    plt.show()

# Generate violin plot for each district
for district_name in districts:
    district_output_path = os.path.join(output_folder, f"processed_buffers_{district_name}.shp")
    district_results = gpd.read_file(district_output_path)
    
    # Ensure CRS consistency
    if district_results.crs != processed_buffers_gdf.crs:
        district_results = district_results.to_crs(processed_buffers_gdf.crs)

    create_violin_plot(district_name, district_results)


In [ ]:
# Step 14: Create Violin Plots for Each District
print("Step 14: Create Violin Plots for Each District")

# Violin Plot with Categorized Tree Fractions
def create_violin_plot(district_name, processed_buffers_gdf):
    # Categorize the tree_frac into 4 quartiles
    processed_buffers_gdf['tree_frac_cat'] = pd.qcut(processed_buffers_gdf['tree_frac'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])

    plt.figure(figsize=(10, 6))
    sns.violinplot(x='tree_frac_cat', y='NO2_Mixed', data=processed_buffers_gdf, cut=0)
    plt.title(f'Violin Plot of Tree Fraction vs. NO2 Data Mixed in {district_name}')
    plt.xlabel('Tree Canopy Coverage Category')
    plt.ylabel('NO2 Data mixed')
    plt.grid(True)

    # Save the plot as PNG
    output_violin_plot_path = os.path.join(output_folder, f"{district_name}_violin_plot_NO2_mixed.png")
    plt.savefig(output_violin_plot_path)
    print(f"Violin Plot saved for {district_name}: {output_violin_plot_path}")

    # Show the plot in Jupyter Notebook
    plt.show()

# Generate violin plot for each district
for district_name in districts:
    district_output_path = os.path.join(output_folder, f"processed_buffers_{district_name}.shp")
    district_results = gpd.read_file(district_output_path)
    
    # Ensure CRS consistency
    if district_results.crs != processed_buffers_gdf.crs:
        district_results = district_results.to_crs(processed_buffers_gdf.crs)

    create_violin_plot(district_name, district_results)

In [ ]:

from sklearn.linear_model import LinearRegression
import pandas as pd

# Calculate Correlation and Regression Coefficient
def calculate_regression(district_name, processed_buffers_gdf):
    tree_column = 'tree_frac_cat'  # For categorized tree fraction
    no2_column = 'NO2_Data'

    correlation = processed_buffers_gdf['tree_frac'].corr(processed_buffers_gdf[no2_column])
    print(f"Correlation for {district_name}: {correlation}")

    # Perform linear regression
    X = processed_buffers_gdf[['tree_frac']].values.reshape(-1, 1)  # Use continuous 'tree_frac' for regression
    y = processed_buffers_gdf[no2_column].values
    model = LinearRegression()
    model.fit(X, y)

    # Print regression coefficient and intercept
    print(f"Regression Coefficient for {district_name}: {model.coef_[0]}")
    print(f"Intercept for {district_name}: {model.intercept_}")

# Calculate correlation and regression for each district
for district_name in districts:
    district_output_path = os.path.join(output_folder, f"processed_buffers_{district_name}.shp")
    district_results = gpd.read_file(district_output_path)
    
    # Ensure CRS consistency
    if district_results.crs != processed_buffers_gdf.crs:
        district_results = district_results.to_crs(processed_buffers_gdf.crs)

    calculate_regression(district_name, district_results)

In [ ]:
from sklearn.linear_model import LinearRegression
import pandas as pd

# Calculate Correlation and Regression Coefficient
def calculate_regression(district_name, processed_buffers_gdf):
    tree_column = 'tree_frac_cat'  # For categorized tree fraction
    no2_column = 'NO2_Mixed'

    correlation = processed_buffers_gdf['tree_frac'].corr(processed_buffers_gdf[no2_column])
    print(f"Correlation for {district_name}: {correlation}")

    # Perform linear regression
    X = processed_buffers_gdf[['tree_frac']].values.reshape(-1, 1)  # Use continuous 'tree_frac' for regression
    y = processed_buffers_gdf[no2_column].values
    model = LinearRegression()
    model.fit(X, y)

    # Print regression coefficient and intercept
    print(f"Regression Coefficient for {district_name}: {model.coef_[0]}")
    print(f"Intercept for {district_name}: {model.intercept_}")

# Calculate correlation and regression for each district
for district_name in districts:
    district_output_path = os.path.join(output_folder, f"processed_buffers_{district_name}.shp")
    district_results = gpd.read_file(district_output_path)
    
    # Ensure CRS consistency
    if district_results.crs != processed_buffers_gdf.crs:
        district_results = district_results.to_crs(processed_buffers_gdf.crs)

    calculate_regression(district_name, district_results)

In [ ]:
import mapclassify

# Interactive Heat Map with Categorized NO2 Data
def create_interactive_heat_map(gdf, column, title, html_output_path):
    # Categorize NO2 data using quantiles
    gdf['NO2_cat'] = pd.qcut(gdf['NO2'], q=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])

    # Create the interactive heatmap
    m = gdf.explore(column='NO2_cat', cmap='YlOrRd', legend=True, tooltip=True)
    m.save(html_output_path)  # Save as HTML file
    return m

# Generate heat map for each district
for district_name in districts:
    district_output_path = os.path.join(output_folder, f"processed_buffers_{district_name}.shp")
    district_results = gpd.read_file(district_output_path)
    
    # Ensure CRS consistency
    if district_results.crs != processed_buffers_gdf.crs:
        district_results = district_results.to_crs(processed_buffers_gdf.crs)

    # Generate and display interactive heat map
    heatmap_path = os.path.join(output_folder, f'{district_name}_heatmap.html')
    create_interactive_heat_map(district_results, 'NO2', f'NO2 Concentration Heat Map - {district_name}', heatmap_path)